<a href="https://colab.research.google.com/github/nsalazard/BICHIN/blob/main/grafo_bichines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
%pip install igraph
%apt-get install libcairo2-dev libjpeg-dev libgif-dev
%pip install pycairo
%pip install scipy


Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%apt-get` not found.


In [82]:
import os
import igraph as ig
import pandas as pd
import numpy as np

EdgeFiles=[]
NodeFiles=[]
for file in os.listdir("./Edges"):
    if file.endswith(".csv"):
        EdgeFiles.append(os.path.join("./Edges", file))
for file in os.listdir("./Nodos"):
    if file.endswith(".csv"):
        NodeFiles.append(os.path.join("./Nodos", file))
        
for i in range(len(EdgeFiles)):
    nodes = pd.read_csv(NodeFiles[i])
    edges = pd.read_csv(EdgeFiles[i]) #Importar datos
   
    edges=edges[edges["weight"]<=50]
   
    g = ig.Graph.TupleList(edges.values,directed=False, #Forma tuplas con las conexiones de los nodos, e indica si usar flechas
                       edge_attrs=edges.columns[2:])#Guarda los atributos extras (peso,tipo,etc)

    nodes_df = nodes.set_index(['id'])

    for column in nodes_df:   
        g.vs[column] = nodes_df.loc[g.vs['name'],column] #g.vs guarda los atributos relacionados con los vertices (nodos)
                                                     #El atributo es el gen-dominante en este caso (Se pueden agregar mas)



    vertex_type_dict = {"F": "blue", "L1": "yellow", "L2": "red","L3":"green","B":"black","R3":"cyan","R2":"magenta","R1":"white"} #Colores asignados a los genes
    visual_style = {}

    visual_style["vertex_size"] = 3 #tamano nodo
    visual_style["vertex_color"] = [vertex_type_dict[type_label] for type_label in g.vs["gen_predominante"]] #asignar color
    #visual_style["vertex_label"] = g.vs["name"] #nombre de los nodos
    visual_style["edge_width"] = [weight/50 for weight in g.es['weight']]
    visual_style["bbox"] = (400, 400)
    visual_style["margin"] = 20
    lay = g.layout("kk")

    out=ig.plot(g, **visual_style,layout=lay)
    out.save("Grapht"+str(i)+".png")

In [18]:
import pandas as pd
import numpy as np
data=pd.read_csv("./poblacion_Uni_100kt1500.csv",delim_whitespace=True)
#data=data.reset_index(drop=False)
#data.columns=["Time","Population"]
data.head(8)

,Time,Population
0,0,100
1,1,100
2,2,100
3,3,100
4,4,100
5,5,100
6,6,100
7,7,100


In [12]:
def my_logistic(t,a,b,c): #b es la rata de cambio de la poblacion y c la capacidad de carga
    return c/(1+a*np.exp(-b*t))

In [4]:
p0=np.random.exponential(size=3)
p0
 #limites para hacer el fit, tenemos maximo 4000 bichitos y cada bichito solo puede dividirse en 2

array([0.45743148, 1.39088241, 0.69482747])

In [19]:
import scipy.optimize as optim
x=np.array(data["Time"])+1
y=np.array(data["Population"])
bounds=(0,[4000,3,4000])
(a,b,c),cov=optim.curve_fit(my_logistic,x,y,bounds=bounds,p0=p0)
a,b,c

(8.881376092625901, 0.006902572402411946, 1433.877964514989)